# 1. Import

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime as dt

PFFScoutingData = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
plays = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
games = pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')

# 2. Setting

In [ ]:
Year = '2018'
StartDate = '2018-10-01' #YYYY-MM-DD
EndDate = '2018-10-30' #YYYY-MM-DD
targetPlayerID = 39975

tStartDate = dt.strptime(StartDate, '%Y-%m-%d')
tEndDate = dt.strptime(EndDate, '%Y-%m-%d')

# 3. Read Tracking Data

In [ ]:
plays = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
Kickoff = plays[plays['specialTeamsPlayType']=='Kickoff']
KickoffReturn = Kickoff[Kickoff['specialTeamsResult']=='Return']
KickoffReturn['Date'] = pd.to_datetime(KickoffReturn['gameId'], format='%Y%m%d%M')
KickoffReturn = KickoffReturn.query('Date > @tStartDate').query('Date < @tEndDate')
# KickoffReturn.to_csv('KickoffReturn.csv')
ReturnerId = KickoffReturn.groupby('returnerId')

tracking = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking%s.csv'%Year).dropna(subset=['time'])
tracking[['Date','Time']] = tracking['time'].str.split('T', expand=True)
tracking['Date'] = pd.to_datetime(tracking['Date'], format='%Y-%m-%d')
tracking = tracking.query('Date > @tStartDate').query('Date < @tEndDate')

players = pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')

#merge
tracking = pd.merge(tracking, KickoffReturn[['gameId', 'playId','absoluteYardlineNumber']], on=['gameId', 'playId'], how='left')

# 4. Arrange Data

In [ ]:
tracking['index'] = tracking.index
dfs = dict()
groupbyReturnerID = KickoffReturn.groupby('returnerId')

df_otherStats = pd.DataFrame(columns=['name','nflId','StartDate','EndDate','count','totalgain','gainAve','afterFirtsContactTotal'])


for nflIdstr, group in ReturnerId:
    if len(nflIdstr) != 5:
        continue
    nflId = int(nflIdstr)

    returnerInfo = tracking.query('nflId == @nflId').groupby(['gameId','playId'])
    dfs[nflId] = pd.DataFrame()
    
    #other Stats
    name = players.query('nflId == @nflId')['displayName'].values[0]
    count = groupbyReturnerID.get_group(nflIdstr).count()['playId']
    totalgain = groupbyReturnerID.get_group(nflIdstr).sum()['kickReturnYardage']
    gainAve = groupbyReturnerID.get_group(nflIdstr).mean()['kickReturnYardage']


    for gameIDandPlayID, info in returnerInfo:
        recieveIndex = info[info['event'] == 'kick_received']['index']
        firstContactIndex = info[info['event'] == 'first_contact']['index']
        tackleIndex = info[info['event'] == 'tackle']['index']
        afterFirtsContactTotal = 0

        #グラフ描画のため座標をいじる
        info.loc[info['absoluteYardlineNumber'] <= 50, 'x'] = -(info['x'] - 120)
        info.loc[info['absoluteYardlineNumber'] <= 50, 'y'] = -(info['y'] - 53.33)

        if recieveIndex.empty:
            continue
        elif dfs[nflId].empty:
            dfs[nflId] = pd.DataFrame(info.query('index >= @recieveIndex.iloc[-1]'))  
        else:
            dfs[nflId] = dfs[nflId].append(info.query('index >= @recieveIndex.iloc[-1]'))

        if firstContactIndex.empty or tackleIndex.empty:
            continue
        else:
            afterFirtsContactTotal += info.query('index == @tackleIndex.iloc[-1]')['x'].iloc[-1] - info.query('index == @firstContactIndex.iloc[-1]')['x'].iloc[-1]
            
    statsSeries = pd.Series([name, nflId, StartDate, EndDate, count, totalgain, gainAve, afterFirtsContactTotal], index=df_otherStats.columns)
    df_otherStats = df_otherStats.append(statsSeries, ignore_index=True)

df_otherStats['totalgainRanking'] = df_otherStats['totalgain'].rank(method='min', ascending=False)
df_otherStats['gainAveRanking'] = df_otherStats['gainAve'].rank(method='min', ascending=False)
df_otherStats['returnerNum'] =len(df_otherStats)

# 5. Plot Returner's Locus

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 12))
# Exclude Touchback
dfs[targetPlayerID] = dfs[targetPlayerID].dropna(subset=['absoluteYardlineNumber'])

groups = dfs[targetPlayerID].groupby('event')

f = groups.get_group('None').plot(x='y', y='x', ax=ax, style='.', label='Run', markersize=7)
f = groups.get_group('first_contact').plot(x='y', y='x', ax=ax, style='o', label='first_contact', markersize=12)
f = groups.get_group('kick_received').plot(x='y', y='x', ax=ax, style='^', label='kick_received', markersize=12)
f = groups.get_group('out_of_bounds').plot(x='y', y='x', ax=ax, style='X', label='out_of_bounds', markersize=12)
f = groups.get_group('tackle').plot(x='y', y='x', ax=ax, style='v', label='tackle', markersize=12)
f = groups.get_group('touchdown').plot(x='y', y='x', ax=ax, style='*', color='red', label='touchdown', markersize=12)
f = groups.get_group('fumble').plot(x='y', y='x', ax=ax, style='*', label='fumble', markersize=12)

xmin = 0
xmax = 53.33
ax.set_xlabel('')
ax.set_xticks([])
ax.set_yticks([])
ax.set_ylim([0,120])
ax.legend()
ax.set_xlim([xmin,xmax])
ax.set_facecolor('green')
ax.patch.set_alpha(0.3)

x = 0.5
ax.text(x,  0,  "0", fontsize=50, color="#ffffff")
ax.text(x, 20, "20", fontsize=50, color="#ffffff")
ax.text(x, 40, "40", fontsize=50, color="#ffffff")
ax.text(x, 60, "60", fontsize=50, color="#ffffff")
ax.text(x, 80, "80", fontsize=50, color="#ffffff")
ax.text(x, 100, "100", fontsize=50, color="#ffffff")
ax.hlines([20,40,60,80,100], xmin, xmax, "white", linestyles='dashed') 